In [11]:
from keras import Sequential
from sklearn.model_selection import train_test_split
from keras.applications import ResNet50
from keras.applications.resnet_v2 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.datasets import cifar10
import pandas as pd
import pickle 

def resnet_accuracies(froze=False, LR_scheduler='Plateau', augmentation=True, batch_size=100, epochs=20, learn_rate=.001):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
#     x_train = preprocess_input(x_train)
#     x_test = preprocess_input(x_test)
    
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3)

    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    y_test = to_categorical(y_test)

    train_generator = ImageDataGenerator(
        rotation_range=2,
        horizontal_flip=True,
        zoom_range=.1)

    val_generator = ImageDataGenerator(
        rotation_range=2,
        horizontal_flip=True,
        zoom_range=.1)

    test_generator = ImageDataGenerator(
        rotation_range=2,
        horizontal_flip=True,
        zoom_range=.1)
    train_generator.fit(x_train)
    val_generator.fit(x_val)
    test_generator.fit(x_test) 
    
    # setting learning rate schedulers
    def scheduler(epoch):
        if epoch < 5:
            return learn_rate 
        if epoch < 20:
            return learn_rate /10
        return learn_rate/100
    if LR_scheduler == 'Custom' :
        callbacks = [LearningRateScheduler(scheduler)]
    elif LR_scheduler == 'Plateau' :
        callbacks = [ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=.01,
        patience=3,
        min_lr=1e-5)]
    else :
        callbacks = [LearningRateScheduler(lambda x: learn_rate)]
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=x_train.shape[1:],
                          classes=y_train.shape[1])

    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Dense(256, activation='relu', input_dim=512))
    model.add(Dropout(.5))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.5))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.layers[0].trainable = not froze

    adam = Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    if augmentation:
        history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size),
                  epochs=epochs, steps_per_epoch=x_train.shape[0] // batch_size,
                  validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps=10,
                  callbacks=callbacks, verbose=1)
    else:
        history = model.fit(x_train, y_train, batch_size=batch_size,
                  epochs=epochs, steps_per_epoch=x_train.shape[0] // batch_size,
                  validation_data=(x_val, y_val), validation_steps=10,
                  callbacks=callbacks, verbose=1)
    return {**history.history, 'test_accuracy': (model.predict(x_test).argmax(axis=1) == y_test.argmax(axis=1)).mean()}



In [12]:
experiments = [{'name': 'Base model',
                'kwargs': {}},
               {'name': 'Frozen',
                'kwargs': {'froze': True}},
               {'name': 'No scheduler',
                'kwargs': {'LR_scheduler': None}},
               {'name': 'Custom scheduler',
                'kwargs': {'LR_scheduler': 'Custom'}},
               {'name': 'No augmentation',
                'kwargs': {'augmentation': False}},
               
               
              ]
for i in range(3):
    assessment = pd.DataFrame([{**resnet_accuracies(**exp['kwargs']),
                               'name': exp['name']} for exp in experiments])
    assessment.to_pickle(f'assessment_{i}.pkl')

Epoch 1/20
350/350 [==============================] - 39s 90ms/step - loss: 2.4161 - accuracy: 0.1283 - val_loss: 1.8292 - val_accuracy: 0.3040
Epoch 2/20
350/350 [==============================] - 30s 86ms/step - loss: 1.7640 - accuracy: 0.3132 - val_loss: 1.6980 - val_accuracy: 0.4040
Epoch 3/20
350/350 [==============================] - 30s 86ms/step - loss: 1.4074 - accuracy: 0.4923 - val_loss: 1.9417 - val_accuracy: 0.4670
Epoch 4/20
350/350 [==============================] - 30s 85ms/step - loss: 1.2068 - accuracy: 0.5927 - val_loss: 1.6043 - val_accuracy: 0.5400
Epoch 5/20
350/350 [==============================] - 30s 85ms/step - loss: 1.0883 - accuracy: 0.6508 - val_loss: 1.7373 - val_accuracy: 0.4770
Epoch 6/20
350/350 [==============================] - 30s 85ms/step - loss: 0.9926 - accuracy: 0.6847 - val_loss: 1.1229 - val_accuracy: 0.6340
Epoch 7/20
350/350 [==============================] - 30s 85ms/step - loss: 0.9421 - accuracy: 0.7058 - val_loss: 1.1261 - val_accuracy: